## Testing Google's flan-t5-xxl model using langchain

Importing necessary libraries and modules

In [41]:
import os
import textwrap
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain import HuggingFaceHub

Set the Hugging Face Hub API token for authentication

In [42]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_API_TOKEN_HERE"

Loading the sample data from a file

In [43]:
text_loader = TextLoader("data.txt")
document = text_loader.load()

Defining a function to split text into lines with a specified width

In [44]:
def split_text_into_lines(text, width=110):
    lines = text.split("\n")
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = "\n".join(wrapped_lines)
    return wrapped_text

Spliting the document into smaller chunks of text

In [45]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
document_chunks = text_splitter.split_documents(document)

Initializing embeddings using Hugging Face models

In [46]:
embeddings = HuggingFaceEmbeddings()

Creating a FAISS (Facebook's AI Similarity Search) vector store from the document chunks and embeddings

In [47]:
vector_store = FAISS.from_documents(document_chunks, embeddings)

Initializing a Hugging Face model from the Hub for question answering

In [48]:
qa_model = HuggingFaceHub(
    repo_id="WizardLM/WizardCoder-Python-34B-V1.0",
    model_kwargs={"temperature": 0.9, "max_length": 512}
)

Load a question answering chain

In [49]:
qa_chain = load_qa_chain(qa_model, chain_type="stuff")

Defining the question for which you want to find answers

In [50]:
question = input("Enter your question: ")

Performing similarity search to retrieve relevant document chunks & running the question answering chain on them

In [51]:
search_results = vector_store.similarity_search(question)
answers = qa_chain.run(input_documents=search_results, question=question)

Printing the typed question and the answer

In [52]:
print(f"Question: {question}")
print(answers)

Question: What is the name of doctor & patient ?
Doctor's name:Dr. Kapoor 
Patient's name:Amit
